# Install the dependencies

In [ ]:
pip install transformers requests beautifulsoup4 pandas numpy bertviz

# Import the needed libraries

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# Initiate the BERT Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment', output_attentions=True)

# Check if the tokens are working and giving correct score

In [ ]:
tokens = tokenizer.encode('It was okayish, could have been better', return_tensors='pt')
result = model(tokens)
attention=result[-1]
result.logits
int(torch.argmax(result.logits))+1

3

# Import bertviz library for visualization

In [ ]:
from bertviz import model_view

# Visualize the above token passed

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(tokens[0]) 
model_view(attention,tokens)

<IPython.core.display.Javascript object>

# Code for YouTube Comments Scrapping

In [ ]:
# -*- coding: utf-8 -*-

# Sample Python code for youtube.commentThreads.list
# See instructions for running these code samples locally:
# https://developers.google.com/explorer-help/code-samples#python

import os

import googleapiclient.discovery

def main():
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"
    DEVELOPER_KEY = ""

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey= DEVELOPER_KEY)

    request = youtube.commentThreads().list(
        part="id,snippet",
        order="relevance",
        maxResults=100,
        videoId="pfqtj3aFBtI"
    )
    response = request.execute()
    return response
  


result=main()
#print(result['pageInfo']['totalResults'])
x=(result['pageInfo']['totalResults'])
result1=[]
for i in range(x):
  result1.append(str(result['items'][i]['snippet']['topLevelComment']['snippet']['textOriginal']) )
print(result1)

['Thanks to Keeps for sponsoring this video! Head to https://keeps.com/Nonstop to get 50% off your first order of Keeps hair loss treatment.', 'Nice video. I would never spend 10.000 dollars for a flight that lasts 8 hours and I find it ridiculous that they charge for WiFi after you paid the price as above.', 'The fact that you can get into trouble for reviewing the airline, says everything you need to know about them lol', "Loved this. I'm an AAdvantage member but avoid AA like the plague, using BA with my miles instead (and paying to fly BA with an AA codeshare, to get the AA miles). I think AA generally *sucks* all around, but I didn't know that when I joined them 17 years ago. In USA air travel, Delta & JetBlue are *MUCH* better than AA. I live in the UK, so BA is fine for me for now. (And Virgin Atlantic.)", 'The $10K price should have been the first red flag for an 8 hour flight.', 'Summing up AA\'s first class service: "I expected nothing and I\'m still let down!"', "For several

# Import the libraries to save the scrapped YouTube Comments

In [ ]:
import numpy as np
import pandas as pd

df = pd.DataFrame(np.array(result1), columns=['reviews'])

# Preview of the dataframe

In [ ]:
df

,reviews
0,Thanks to Keeps for sponsoring this video! Hea...
1,Nice video. I would never spend 10.000 dollars...
2,The fact that you can get into trouble for rev...
3,Loved this. I'm an AAdvantage member but avoid...
4,The $10K price should have been the first red ...
5,"Summing up AA's first class service: ""I expect..."
6,For several decades now I have refused to fly ...
7,"As a rule, I avoid US carriers when I fly inte..."
8,“AA barely advertise their first class at all”...
9,"Thanks for the warning, Josh. I’m sorry your f..."


# Function to calculate the sentiment score of a review

In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

sentiment_score(df['reviews'].iloc[0])

1

# To apply the above function on each of the review

In [ ]:
df['sentiment'] = df['reviews'].apply(lambda x: sentiment_score(x[:512]))

# Preview of dataframe with their sentiment scores

In [ ]:
df

,reviews,sentiment
0,Thanks to Keeps for sponsoring this video! Hea...,1
1,Nice video. I would never spend 10.000 dollars...,4
2,The fact that you can get into trouble for rev...,5
3,Loved this. I'm an AAdvantage member but avoid...,5
4,The $10K price should have been the first red ...,3
5,"Summing up AA's first class service: ""I expect...",3
6,For several decades now I have refused to fly ...,5
7,"As a rule, I avoid US carriers when I fly inte...",2
8,“AA barely advertise their first class at all”...,2
9,"Thanks for the warning, Josh. I’m sorry your f...",4


# Caculating the average sentiment score

In [ ]:
average_sentiment=(sum(df['sentiment']))/(len(df['sentiment']))

In [ ]:
average_sentiment

2.95

# Code to scrape the restaurant reviews

In [ ]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews1 = [result.text for result in results]

# Saving the above scrapped reviews in a new DataFrame

In [ ]:
df1 = pd.DataFrame(np.array(reviews1), columns=['reviews'])
df1['reviews'].iloc[0]
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

sentiment_score(df1['reviews'].iloc[1])

4

# Applying the same sentiment score function on each of the review

In [ ]:
df1['sentiment'] = df1['reviews'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
df1

,reviews,sentiment
0,Great coffee and vibe. That's all you need. C...,5
1,Great coffee and vibe. That's all you need. C...,4
2,I came to Social brew cafe for brunch while ex...,5
3,Ricotta hot cakes! These were so yummy. I ate ...,5
4,I went here a little while ago- a beautiful mo...,2
5,We came for brunch twice in our week-long visi...,4
6,Ron & Jo are on the go down under and Wow! We...,5
7,Good coffee and toasts. Straight up and down -...,5
8,This place is a gem. The ambiance is to die fo...,3
9,Delicious. The waitress was hot. The burger wa...,4


# Calculating the average sentiment score

In [ ]:
average_sentiment1=(sum(df1['sentiment']))/(len(df1['sentiment']))

In [ ]:
average_sentiment1

4.2727272727272725